In [3]:
import os, json, datetime, sys
from azureml.core import Workspace, Environment, Datastore
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AksWebservice, AciWebservice, Webservice
from azureml.core.environment import Environment

workspace = Workspace.from_config()

In [4]:
import json
with open("./scripts/metric.json") as f:
    metric = json.load(f)
metric

{'run_id': 'arima-mlops-remote-v3_1650758172_65c70524',
 'model_name': 'arima-model-v3',
 'RMSE': 461.5953678665711,
 'R2': 0.10350758763675871}

In [5]:
model_path = os.path.join('./', 'models')
model = Model.register(model_path = model_path, # this points to a local file
                       model_name = metric['model_name'], # this is the name the model is registered as
                       tags = metric,
                       description = "arima model",
                       workspace = workspace)

print(model.name, model.description, model.version)

Registering model arima-model-v3
arima-model-v3 arima model 7


In [6]:
%%writefile score.py
import pickle
import os, json
import numpy
from azureml.core.model import Model
from statsmodels.tsa.arima_model import ARIMA


def init():
    global model
    import joblib

    # load the model from file into a global object
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'models/arima_model.pkl')
    # model_path = Model.get_model_path(model_name="arima-model")
    model = joblib.load(model_path)


def run(raw_data):
    try:
        data = json.loads(raw_data)["data"]
        data = numpy.array(data)
        result=model.forecast(steps=data[0])[0]
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Overwriting score.py


In [7]:
myenv = Environment.from_conda_specification(name="arima-env", file_path="./arima-env.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=3, 
                                               tags={'name': metric['model_name'], 'framework': 'statsmodels'},
                                               description='arima inference')

service = Model.deploy(workspace=workspace,
                           name=metric['model_name'], 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig, overwrite=True)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-24 00:02:45+00:00 Creating Container Registry if not exists.
2022-04-24 00:02:45+00:00 Registering the environment.
2022-04-24 00:02:46+00:00 Use the existing image.
2022-04-24 00:02:46+00:00 Generating deployment configuration.
2022-04-24 00:02:47+00:00 Submitting deployment to compute.
2022-04-24 00:02:53+00:00 Checking the status of deployment arima-model-v3..
2022-04-24 00:04:45+00:00 Checking the status of inference endpoint arima-model-v3.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [8]:
print(service.get_logs())

2022-04-24T00:04:24,527867800+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-04-24T00:04:24,530129300+00:00 - rsyslog/run 
2022-04-24T00:04:24,543682200+00:00 - iot-server/run 
2022-04-24T00:04:24,585013200+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-04-24T00:04:24,973842400+00:00 - iot-server/finish 1 0
2022-04-24T00:04:24,976693600+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (73)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 100
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2022-04-24 00:04:30,045 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2022-04-24 00:04:30,046 | root | INFO | Starting up request id generator
2022-04-24 00:04:30,046 | root | INFO | Sta

In [9]:
print(service.scoring_uri)

http://a021201f-b48a-4cf1-974b-b270ede0dae3.japaneast.azurecontainer.io/score


In [10]:
step_size=[3]
test_sample = json.dumps({"data": step_size})
test_sample = bytes(test_sample, encoding="utf8")
print(test_sample)

b'{"data": [3]}'


In [11]:
prediction = service.run(input_data=test_sample)
print(prediction)

{"result": [626.5655200695544, 768.0510945310739, 819.0250224534511]}
